# Introduction to this Notebook

This Jupyter Notebook encompassess a series of scripts written in Python by Daniel Teixeira dos Santos, a Data Community Innovator at the Data Community of Practice ([link to my forum account](https://rcop.michaeljfox.org/u/danieltds/summary)). These scripts were written using data from PPMI, obtained through LONI. These files are linked to the MJFF Research Community's GitHub repository ([link here](https://github.com/MJFF-ResearchCommunity/Useful-PPMI-Clinical-Codes))

The goal of these scripts is to provide researchers some relevant clinical data that are extracted in a meaningful way form the data that is already available in PPMI. All the necessary input datasets can be obtained [here](https://ida.loni.usc.edu/pages/access/studyData.jsp?project=PPMI) after applying for registration for access to the PPMI data. All outputs from the analyses were removed to comply with privacy and data sharing principles. Some of these scripts were developed with the help of AI tools such as ChatGPT 4o.

This analysis requires two different folders to exist within the main folder. Those are "data" and "priv". The "data" folder is the place where you should store your datasets downloaded from LONI. The priv folder is the one the results will be exported to. These folders will be generated automatically at the beginning of this script, if they don't exist.

# Importing and Setting Paths

In [ ]:
import os
import pandas as pd
import numpy as np
import math
import glob
import warnings

# Automatically find the "Useful PPMI Clinical Codes" directory
CURRENT_DIR = os.getcwd()
while not CURRENT_DIR.endswith("Useful PPMI Clinical Codes") and os.path.dirname(CURRENT_DIR) != CURRENT_DIR:
    CURRENT_DIR = os.path.dirname(CURRENT_DIR)

BASE_DIR = CURRENT_DIR

# Define paths for "data" and "report" directories
DATA_DIR = os.path.join(BASE_DIR, "data")
PRIV_DIR = os.path.join(BASE_DIR, "priv")

# Ensure both directories exist, create them if not
for directory in [DATA_DIR, PRIV_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created missing folder: {directory}")
    else:
        print(f"Found folder: {directory}")

# Ignore persistent warnings
warnings.simplefilter("ignore", UserWarning)

# Configure Pandas for better data visualization
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = "{:,.3f}".format

# List available files in both directories
print("Files in data directory:", os.listdir(DATA_DIR))
print("Files in priv directory:", os.listdir(PRIV_DIR))


# Levodopa responsiveness

Levodopa responsiveness is a very interesting marker that different studies have approached (example: https://pubmed.ncbi.nlm.nih.gov/38898616/). The PPMI protocol states that, whenever possible, patients should be evaluated both in the OFF and ON states. In that way, we can calculate levodopa challenge responses by using the formula: response = (off - on) / off x 100. A good levodopa response is (usually at least 30%), for example, a pre-requisite for DBS surgery.

**Necessary PPMI datasets:** MDS-UPDRS Part III Treatment Determination and Part III: Motor Examination

**Last Update:** February 9, 2025

## Organizing

### General read and view

In [ ]:
MDS3 = pd.read_csv(os.path.join(DATA_DIR, "MDS-UPDRS_Part_III_09Feb2025.csv"))
print('Lenght of the dataset:', len(MDS3))
MDS3.head()

In [ ]:
# Define concepts with their corresponding columns
# This can be useful to calculate characteristic-specific levodopa responses
concepts = {
    'Rigidity': ["NP3RIGLL", "NP3RIGLU", "NP3RIGN", "NP3RIGRL", "NP3RIGRU"], # 3.3 (all elements)
    "Tremor": ["NP3KTRML", "NP3KTRMR", "NP3PTRML", "NP3PTRMR", "NP3RTALJ", "NP3RTALL", "NP3RTALU", "NP3RTARL", "NP3RTARU", "NP3RTCON"], # 3.15 + 3.16 + 3.17 + 3.18
    "Gait_and_Posture": ["NP3RISNG", "NP3GAIT", "NP3FRZGT", "NP3PSTBL", "NP3POSTR"], # 3.9 + 3.10 + 3.11 + 3.12 + 3.13
    "Bradykinesia": ["NP3FTAPR", "NP3FTAPL", "NP3HMOVR", "NP3HMOVL", "NP3PRSPR", "NP3PRSPL", "NP3TTAPR", "NP3TTAPL", "NP3LGAGR", "NP3LGAGL", "NP3BRADY"],# 	3.4 + 3.5 + 3.6 + 3.7 + 3.8 + 3.14
    'All_MDS3': ["NP3SPCH", "NP3FACXP", "NP3RIGN", "NP3RIGRU", "NP3RIGLU", "NP3RIGRL", "NP3RIGLL",
    "NP3FTAPR", "NP3FTAPL", "NP3HMOVR", "NP3HMOVL", "NP3PRSPR", "NP3PRSPL", "NP3TTAPR",
    "NP3TTAPL", "NP3LGAGR", "NP3LGAGL", "NP3RISNG", "NP3GAIT", "NP3FRZGT", "NP3PSTBL",
    "NP3POSTR", "NP3BRADY", "NP3PTRMR", "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU",
    "NP3RTALU", "NP3RTARL", "NP3RTALL", "NP3RTALJ", "NP3RTCON"]}

In [ ]:
# Function to compute the sum, considering NaN
def sum_with_nan(series):
    if series.isna().any():
        return np.nan
    else:
        return series.sum()

concepts_list = []

# Add new columns with the sum of values for each concept
for concept, columns in concepts.items():
    sum_column = concept
    concepts_list.append(sum_column)
    MDS3[sum_column] = MDS3[columns].apply(sum_with_nan, axis=1)

In [ ]:
# Let's check the overall distribution of the PDSTATE (situation in which the patient was examined)
MDS3['PDSTATE'].value_counts()

Check how many different entries (more than one for patient) have at least 2 MDS evaluations (one in the OFF and one in the ON states)

In [ ]:
# Group by PATNO and EVENT_ID and filter groups with more than one entry
duplicates = MDS3.groupby(['PATNO', 'EVENT_ID']).filter(lambda x: len(x) > 1)

# Extract unique PATNO and EVENT_ID pairs with values in the specified columns
result_test = duplicates.groupby(['PATNO', 'EVENT_ID']).agg({
    'PDSTATE': lambda x: tuple(x), # Which functional state is the participant currently in?
    'HRPOSTMED': lambda x: tuple(x), # Hours between last dose of PD medication and NUPDRS3 exam
    'EXAMTM': lambda x: tuple(x), # Time of NUPDRS3 exam
    'HRDBSON': lambda x: tuple(x), # Hours between DBS device turned on and NUPDRS3 exam
    'DBSYN': lambda x: tuple(x), # Does participant have DBS
    'ONOFFORDER': lambda x: tuple(x), # First Part III exam OFF or ON
    'OFFEXAM': lambda x: tuple(x), # OFF exam performed
    'OFFNORSN': lambda x: tuple(x), # Reason OFF exam not performed
    'DBSOFFTM': lambda x: tuple(x), # Time DBS turned off before OFF exam
    'ONEXAM': lambda x: tuple(x), # ON exam performed
    'ONNORSN': lambda x: tuple(x), # Reason ON exam not performed
    'DBSONTM': lambda x: tuple(x), # Time DBS turned on before ON exam
    'PDMEDDT': lambda x: tuple(x), # Date of most recent PD med dose before exam
    'PDMEDTM': lambda x: tuple(x), # Time of most recent PD med dose before exam
}).reset_index()

print('Lenght of the dataset considering at least 2 evaluations:', len(result_test))
result_test.head()

Check how many different entries (more than one for patient) have at least 3 MDS evaluations (one in the OFF and one in the ON states).

This is unusual, but the number is low. Did this just to check how data are displayed.

In [ ]:
# Group by PATNO and EVENT_ID and filter groups with more than one entry
duplicates = MDS3.groupby(['PATNO', 'EVENT_ID']).filter(lambda x: len(x) > 2)

# Extract unique PATNO and EVENT_ID pairs with values in the specified columns
result_test = duplicates.groupby(['PATNO', 'EVENT_ID']).agg({
    'PDSTATE': lambda x: tuple(x), # Which functional state is the participant currently in?
    'HRPOSTMED': lambda x: tuple(x), # Hours between last dose of PD medication and NUPDRS3 exam
    'HRDBSON': lambda x: tuple(x), # Hours between DBS device turned on and NUPDRS3 exam
    'DBSYN': lambda x: tuple(x), # Does participant have DBS
    'ONOFFORDER': lambda x: tuple(x), # First Part III exam OFF or ON
    'OFFEXAM': lambda x: tuple(x), # OFF exam performed
    'OFFNORSN': lambda x: tuple(x), # Reason OFF exam not performed
    'DBSOFFTM': lambda x: tuple(x), # Time DBS turned off before OFF exam
    'ONEXAM': lambda x: tuple(x), # ON exam performed
    'ONNORSN': lambda x: tuple(x), # Reason ON exam not performed
    'DBSONTM': lambda x: tuple(x), # Time DBS turned on before ON exam
    'PDMEDDT': lambda x: tuple(x), # Date of most recent PD med dose before exam
    'PDMEDTM': lambda x: tuple(x), # Time of most recent PD med dose before exam
}).reset_index()

print('Lenght of the dataset considering more than 2 evaluations:', len(result_test))
result_test.head()

### Missing correction

Missing values in the MDS-UPDRS can be written as 101 (Unable to Rate). Let's identify those and treat them as NaN for our analysis not to be biased by those high numbers

In [ ]:
# Dataset has some "101", which are "Unable to Rate"
MDS3['NP3RIGLL'].value_counts(dropna=False)

In [ ]:
# Converting unables to rate to nan
MDS3 = MDS3.replace(101,np.nan)

# Forcing to become float, ignore errors
MDS3 = MDS3.apply(pd.to_numeric, errors='ignore')

# Checking
MDS3['NP3RIGLL'].value_counts(dropna=False)

### DBS categories

Patients also undergo DBS as this is informed in the MDS3 scale. We will want, in this code, generate separate values for evaluations that had a DBS and evaluations without a DBS

In [ ]:
print('Length of the dataset before removing DBS:', len(MDS3))
MDS3_nodbs = MDS3[MDS3['DBSYN'].isin([0, np.nan])].reset_index(drop=True)
print('Length of the dataset after removing DBS:', len(MDS3_nodbs))

In [ ]:
print('Length of the dataset before subsetting by DBS:', len(MDS3))
MDS3_dbs = MDS3[MDS3['DBSYN'].isin([1])].reset_index(drop=True)
print('Length of the dataset after subsetting by DBS:', len(MDS3_dbs))

## Calculating response (no DBS)

In [ ]:
# Group by PATNO and EVENT_ID and filter groups with more than one entry
duplicates = MDS3_nodbs.groupby(['PATNO', 'EVENT_ID']).filter(lambda x: len(x) == 2)

# Extract unique PATNO and EVENT_ID pairs with values in the specified columns
result = duplicates.groupby(['PATNO', 'EVENT_ID']).agg({
    'PDSTATE': lambda x: tuple(x), # Which functional state is the participant currently in?
    'HRPOSTMED': lambda x: tuple(x), # Hours between last dose of PD medication and NUPDRS3 exam
    'ONOFFORDER': lambda x: tuple(x), # First Part III exam OFF or ON
    'OFFEXAM': lambda x: tuple(x), # OFF exam performed
    'ONEXAM': lambda x: tuple(x), # ON exam performed
    'Rigidity': lambda x: tuple(x), # Rigidity
    'Tremor': lambda x: tuple(x), # Tremor
    'Gait_and_Posture': lambda x: tuple(x), # Gait and Posture
    'Bradykinesia': lambda x: tuple(x), # Bradykinesia
    'All_MDS3': lambda x: tuple(x), # MDS_3 Complete
}).reset_index()

print('Lenght of the dataset considering exactly 2 evaluations:', len(result))
result.head()

Main function to calculate the response. It uses the tuples of OFF and ON to calculate these responses, then generates new columns detailing the actual responses. There are also other columns that helps us better understaning what is happening, such as "Time_since_levodopa", which helps us be sure that the responses are correctly calculating OFF versus ON responses, and not the opposite.

The lowest HRPOSTMED (hours after last medication) is used to define the ON state

In [ ]:
# Define the function to calculate the response
def calculate_response(on, off):
    if pd.isna(on) or pd.isna(off) or off == 0:
        return np.nan
    return ((off - on) / off) * 100

# Define a function to round values and handle errors
def round_to_int(value):
    try:
        return int(round(value))
    except (ValueError, TypeError):
        return np.nan

# Function to process the dataset
def process_dataset(df, concepts_list):
    new_columns = []
    df['Time_since_levodopa'] = df['HRPOSTMED'].apply(lambda x: min(x) if not any(pd.isna(v) for v in x) else np.nan) * 60  # Convert to minutes
    df['Time_since_levodopa'] = df['Time_since_levodopa'].apply(round_to_int)
    for var in concepts_list:
        new_col_name = f'{var}_resp'
        new_columns.append(new_col_name)
        df[new_col_name] = df.apply(lambda row: calculate_response(
            row[var][0] if not any(pd.isna(v) for v in row['HRPOSTMED']) and row['HRPOSTMED'][0] < row['HRPOSTMED'][1] else row[var][1],  # on
            row[var][1] if not any(pd.isna(v) for v in row['HRPOSTMED']) and row['HRPOSTMED'][0] < row['HRPOSTMED'][1] else row[var][0]   # off
        ) if not any(pd.isna(v) for v in row['HRPOSTMED']) else np.nan, axis=1)
    return df, new_columns

# Run the function
levodopa_response, newcols = process_dataset(result, concepts_list)

# Remove those that don't have info on time since levodopa
filtered_df = levodopa_response[levodopa_response['Time_since_levodopa'].notna()]

# Count the unique values in the 'Time_since_levodopa' column
unique_patients_levodopa = filtered_df['Time_since_levodopa'].nunique()
print('Length of entire dataset:', len(levodopa_response))
print('Length of subsetted dataset:', len(filtered_df))
print('Number of unique patients NOT using DBS that took levodopa in a challenge:', unique_patients_levodopa)

# Display the processed DataFrame
levodopa_response.head()

There are some negative values, however, they are a minority of the data. Some of the most significant ones can be typos, and other just a slight paradoxical worsening / variation due to clinician's judgment

In [ ]:
levodopa_response[newcols].describe(include='all')

In [ ]:
# First, let's have a general idea of some patients with negative responses
print(len(levodopa_response[levodopa_response['All_MDS3_resp'] < 0]))
levodopa_response[levodopa_response['All_MDS3_resp'] < 0].head(10)[['PDSTATE','HRPOSTMED','All_MDS3','All_MDS3_resp']]

In [ ]:
# Second, let's have a general idea of some patients with EXTREME negative responses
print(len(levodopa_response[levodopa_response['All_MDS3_resp'] < -100]))
levodopa_response[levodopa_response['All_MDS3_resp'] < -100].head(10)[['PDSTATE','HRPOSTMED','All_MDS3','All_MDS3_resp']]

As you can see, most patients with responses between -100 and 0 have just a mild paradoxical response or variation due to clinical judgment. However, most patients < -100 probably have typos that invalidated the analysis. Fortunately, thery are only a few.

I will not remove those, but **I highly suggest you take that into account in your analysis**

Exporting

In [ ]:
# Exporting
levodopa_response.to_csv(os.path.join(PRIV_DIR, "Levodopa_challenge_no_DBS.csv"), index=False)

## Calculating response (DBS)

In [ ]:
# Group by PATNO and EVENT_ID and filter groups with more than one entry
duplicates = MDS3_dbs.groupby(['PATNO', 'EVENT_ID']).filter(lambda x: len(x) == 2)

# Extract unique PATNO and EVENT_ID pairs with values in the specified columns
result = duplicates.groupby(['PATNO', 'EVENT_ID']).agg({
    'PDSTATE': lambda x: tuple(x), # Which functional state is the participant currently in?
    'HRPOSTMED': lambda x: tuple(x), # Hours between last dose of PD medication and NUPDRS3 exam
    'EXAMTM': lambda x: tuple(x), # Time of NUPDRS3 exam
    'ONOFFORDER': lambda x: tuple(x), # First Part III exam OFF or ON
    'OFFEXAM': lambda x: tuple(x), # OFF exam performed
    'ONEXAM': lambda x: tuple(x), # ON exam performed
    'DBSYN': lambda x: tuple(x), # Does participant have DBS
    'DBSOFFTM': lambda x: tuple(x), # Time DBS turned off before OFF exam
    'DBSONTM': lambda x: tuple(x), # Time DBS turned on before ON exam
    'HRDBSON': lambda x: tuple(x), # Hours between DBS device turned on and NUPDRS3 exam
    'Rigidity': lambda x: tuple(x), # Rigidity
    'Tremor': lambda x: tuple(x), # Tremor
    'Gait_and_Posture': lambda x: tuple(x), # Gait and Posture
    'Bradykinesia': lambda x: tuple(x), # Bradykinesia
    'All_MDS3': lambda x: tuple(x), # MDS_3 Complete
}).reset_index()

print('Lenght of the dataset considering exactly 2 evaluations:', len(result))
result.head()

In [ ]:
# Run the function
levodopa_response, newcols = process_dataset(result, concepts_list)

# Remove those that don't have info on time since levodopa
filtered_df = levodopa_response[levodopa_response['Time_since_levodopa'].notna()]

# Count the unique values in the 'Time_since_levodopa' column
unique_patients_levodopa = filtered_df['Time_since_levodopa'].nunique()
print('Length of entire dataset:', len(levodopa_response))
print('Length of subsetted dataset:', len(filtered_df))
print('Number of unique patients using DBS that took levodopa in a challenge:', unique_patients_levodopa)

# Showing the subsetted dataset
filtered_df.head()

In [ ]:
# Exporting
levodopa_response.to_csv(os.path.join(PRIV_DIR, "Levodopa_challenge_DBS.csv"), index=False)